In [1]:
# visualize cell in `modal_training/modal_results/single_modal.xyz` using the plot atoms func from plot_utils.py
import sys
!{sys.executable} -m pip install nbformat==5.1.0

  Using cached nbformat-5.1.0-py3-none-any.whl.metadata (1.4 kB)
Reason for being yanked: Missing schema files
Using cached nbformat-5.1.0-py3-none-any.whl (113 kB)
  Attempting uninstall: nbformat
    Found existing installation: nbformat 4.2.0
    Uninstalling nbformat-4.2.0:
      Successfully uninstalled nbformat-4.2.0

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
import sys
import nbformat
print(f"nbformat version: {nbformat.__version__}")
print(f"Python executable: {sys.executable}")

nbformat version: 5.1.0
Python executable: /Users/kosta/Documents/Research/martiniani/omatg_TomEgg/code/ghosting-repo/.venv/bin/python


In [3]:
import sys
from pathlib import Path
import torch
import numpy as np
from ase.io import read

# Add ghosting-repo to Python path (for plot_utils)
PROJECT_ROOT = Path("/Users/kosta/Documents/Research/martiniani/omatg_TomEgg")
GHOSTING_REPO_DIR = PROJECT_ROOT / "code" / "ghosting-repo"
if str(GHOSTING_REPO_DIR) not in sys.path:
    sys.path.insert(0, str(GHOSTING_REPO_DIR))

from plot_utils import plot_crystal_with_points


class SimpleData:
    """Simple data class compatible with plot_crystal_with_points"""
    def __init__(self):
        self.cell = None
        self.pos = None
        self.species = None


def ase_atoms_to_simple_data(atoms):
    """Convert ASE Atoms object to SimpleData format expected by plot_crystal_with_points"""
    species_np = atoms.numbers.astype("int64")
    ghost_mask = np.zeros_like(species_np, dtype=bool)

    if "is_ghost" in atoms.arrays:
        ghost_mask |= atoms.get_array("is_ghost").astype(bool)

    if "ghost_atomic_number" in atoms.arrays:
        ghost_values = np.unique(atoms.get_array("ghost_atomic_number")).astype("int64")
        ghost_mask |= np.isin(species_np, ghost_values)

    if not ghost_mask.any():
        ghost_mask |= species_np <= 0

    species = torch.from_numpy(species_np)
    if ghost_mask.any():
        species = species.masked_fill(torch.from_numpy(ghost_mask), -1)

    data = SimpleData()
    data.pos = torch.from_numpy(atoms.positions).to(dtype=torch.float64)
    data.species = species.long()
    data.cell = torch.from_numpy(atoms.cell.array).to(dtype=torch.float64).unsqueeze(0)
    return data

In [8]:
# Load the xyz file
xyz_path = GHOSTING_REPO_DIR / "modal_training" / "modal_results" / "single_res_20251202_2000.xyz"
atoms_list = read(str(xyz_path), index=":")
print(f"Loaded {len(atoms_list)} structures from {xyz_path}")

print(len(atoms_list))
# Visualize the first structure (change index to visualize different structures)
sample_idx = 0
sample_data = ase_atoms_to_simple_data(atoms_list[sample_idx])
plot_crystal_with_points(
    sample_data, 
    sample_idx, 
    title=f"single_modal structure {sample_idx}"
)

Loaded 1 structures from /Users/kosta/Documents/Research/martiniani/omatg_TomEgg/code/ghosting-repo/modal_training/modal_results/single_res_20251202_2000.xyz
1


In [9]:
# Load the xyz file
xyz_path = GHOSTING_REPO_DIR / "modal_training" / "modal_results" / "single_modal_20251202_1900_single.xyz"
atoms_list = read(str(xyz_path), index=":")
print(f"Loaded {len(atoms_list)} structures from {xyz_path}")

print(len(atoms_list))
# Visualize the first structure (change index to visualize different structures)
sample_idx = 0
sample_data = ase_atoms_to_simple_data(atoms_list[sample_idx])
plot_crystal_with_points(
    sample_data, 
    sample_idx, 
    title=f"single_modal structure {sample_idx}"
)

Loaded 1 structures from /Users/kosta/Documents/Research/martiniani/omatg_TomEgg/code/ghosting-repo/modal_training/modal_results/single_modal_20251202_1900_single.xyz
1
